In [1]:
import bpy
import numpy as np
from IPython.display import display, Image
import os
import mathutils
from tqdm import tqdm
from typing import List, Dict
import bmesh

import scipy.sparse
import scipy.linalg

import matplotlib.pyplot as plt

from line_profiler import profile

# install packages : "C:\Users\Pierre.Gilibert\OneDrive - ARVERNE\Bureau\blender-4.3.1-windows-x64\4.3\python\bin\python.exe" -m pip install scipy 

In [2]:
FOLDER = "C:\\Users\\pierr\\Documents\\Blender\\GeometryProcessing_Python\\"
# FOLDER = "C:\\Users\\Pierre.Gilibert\\OneDrive - ARVERNE\\Documents\\Divers\\blender"
bpy.ops.wm.open_mainfile(filepath=os.path.join(FOLDER, "blender_notebook_v01.blend"))

{'FINISHED'}

In [3]:
# def cross_product(x, y):
#     return np.array([
#         x[1]*y[2] - x[2]*y[1],
#         x[2]*y[0] - x[0]*y[2],
#         x[0]*y[1] - x[1]*y[0]
#     ])

# def dot_product(x,y):
#     return x[0]*y[0] + x[1]*y[1] + x[2]*y[2]

# def rotate_coord_system(u,v, nf, n=None):
#     """
#     rotate the vectors u and v to the plane of normal nf
#     Args:
#         u (np.array (N,3)) : 1st set of vector to be rotated
#         v (np.array (N,3)) : 2nd set of vector to be rotated
#         nf (np.array (N,3)) : normal vectors of the plane to rotate into
#         n (np.array (N,3)) : cross_product(u,v)
#     Output:
#         new_u, new_v (np.array(N, 3)) : rotated u and v (orthogonal to nf)
#     """
#     # n = np.cross(u,v)
#     if n is None:
#         n = cross_product(u, v)
#         n = n/np.linalg.norm(n)

#     # dot = np.dot(nf,n)
#     dot = dot_product(nf, n)
#     if dot <= -1:
#         return -u,-v
    
#     perp = nf - dot*n
#     dperp = (nf + n)/(1+dot)
#     new_u = u - dperp * dot_product(perp,u) # u - dperp * (perp.u)
#     new_v = v - dperp * dot_product(perp,v)

#     return new_u, new_v

def rotate_coord_system_vectorized(uv, vv, nf, nv):
    dot = np.sum(nv*nf, axis=1)
    dot_neg = np.where(dot <= -1)[0]
    dot = dot[:,None]
    perp = nf - dot * nv
    dperp = (nf + nv)/(1+dot)
    new_uv = uv - dperp * np.sum(perp*uv, axis=1)[:,None] # u - dperp * (perp.u)
    new_vv = vv - dperp * np.sum(perp*vv, axis=1)[:,None]

    new_uv[dot_neg]=-uv[dot_neg]
    new_uv[dot_neg]=-vv[dot_neg]

    return new_uv, new_vv

def project_curvature_tensor_vectorized(uf, vf, nf, old_ku, old_kuv, old_kv, up, vp, n):
    """
    Perform a projection of the tensor variables to the vertex coordinate system.
    
    Parameters:
    uf, vf (numpy.ndarray): Face coordinate system unit vectors.
    nf (numpy.ndarray): Normal vector of the face.
    old_ku, old_kuv, old_kv (float): Face curvature tensor variables.
    up, vp (numpy.ndarray): Vertex coordinate system unit vectors.
    n (numpy.ndarray): Normal vector of the vertex.
    
    Returns:
    new_ku, new_kuv, new_kv (float): Vertex curvature tensor variables.
    """
    def compute_coeff(a, b, ku, kuv, kv, c, d):
        return ku * c * a + kuv * (a*d + b*c) + kv * d * b # [a,b] @ [[ku, kuv], [kuv, kv]] @ [[c], [d]]
    
    # Rotate the coordinate system
    r_new_u, r_new_v = rotate_coord_system_vectorized(up, vp, nf, n)

    u1, u2 = np.sum(r_new_u*uf, axis=1), np.sum(r_new_v*uf, axis=1)
    v1, v2 = np.sum(r_new_u*vf, axis=1), np.sum(r_new_v*vf, axis=1)

    new_ku = compute_coeff(u1, v1, old_ku, old_kuv, old_kv, u1, v1)
    new_kuv = compute_coeff(u1, v1, old_ku, old_kuv, old_kv, u2, v2)
    new_kv = compute_coeff(u2, v2, old_ku, old_kuv, old_kv, u2, v2)

    return new_ku, new_kuv, new_kv

# def project_curvature_tensor(uf, vf, nf, old_ku, old_kuv, old_kv, up, vp, n):
#     """
#     Perform a projection of the tensor variables to the vertex coordinate system.
    
#     Parameters:
#     uf, vf (numpy.ndarray): Face coordinate system unit vectors.
#     nf (numpy.ndarray): Normal vector of the face.
#     old_ku, old_kuv, old_kv (float): Face curvature tensor variables.
#     up, vp (numpy.ndarray): Vertex coordinate system unit vectors.
#     n (numpy.ndarray): Normal vector of the vertex.
    
#     Returns:
#     new_ku, new_kuv, new_kv (float): Vertex curvature tensor variables.
#     """
    
#     # Rotate the coordinate system
#     r_new_u, r_new_v = rotate_coord_system(up, vp, nf, n)
    
#     # Curvature tensor matrix for the face
#     old_tensor = np.array([[old_ku, old_kuv], 
#                            [old_kuv, old_kv]])
    
#     # Project coordinates
#     u1 = np.dot(r_new_u, uf)
#     v1 = np.dot(r_new_u, vf)
#     u2 = np.dot(r_new_v, uf)
#     v2 = np.dot(r_new_v, vf)

#     # Calculate new curvature tensor variables
#     new_ku = np.dot([u1, v1], np.dot(old_tensor, [u1, v1]))
#     new_kuv = np.dot([u1, v1], np.dot(old_tensor, [u2, v2]))
#     new_kv = np.dot([u2, v2], np.dot(old_tensor, [u2, v2]))

#     return new_ku, new_kuv, new_kv

class MYMesh:
    def __init__(self):
        self.bm = bmesh.new()
        self.facecorners = []
        self.vert2facecorner = {}
        self.facecorner_attributes = {}             # facecorners attributes
        self.vertex_attributes = {"u":{}, "v":{}}   # u,v : basis vector of the tangent plane at each vertex
        self.face_attributes = {}
        self.face_vertex_to_facecorner = {}
        self.co = None                              # A (|V|, 3) np array where row i contains the x,y,z coordinates of the vertex indexed by i 
        self.cotan = None                           # A (3|F|,) np array where entry i is the cotangent of the angle of facecorner (loop) i
        self.face_areas = None                      # A (|F|,) np array where entry i is the area for face i
        self.internal_angles = None                 # A (3|F|,) np array where entry i is the angle of facecorner (loop) i
        self.fv = None                              # A (|F|, 3) np array where row i contains the indices of the vertices of face i


    def from_mesh(self, mesh_data):
        """ Mimic the bmesh from_mesh function. """
        self.bm.from_mesh(mesh_data)
        self.vert2facecorner = {v:[] for v in self.verts}
        self.facecorners = []
        self.facecorner_attributes["area"] = {}
        for face in self.faces:
            self.face_vertex_to_facecorner[face] = {}
            for loop in face.loops:
                self.vert2facecorner[loop.vert].append(loop)
                self.facecorners.append(loop)
                self.facecorner_attributes["area"][loop] = 0 # init 0 area
                self.face_vertex_to_facecorner[face][loop.vert] = loop
        self.fv = np.array([[fc.vert.index for fc in f.loops] for f in self.faces], dtype=int) # face #f has vertes [vi, vj, vk]
        

    def to_mesh(self, mesh_data):
        """ Mimic the bmesh to_mesh function. """
        self.bm.to_mesh(mesh_data)
    
    # Add any other bmesh methods as needed:
    def free(self):
        """ Mimic the bmesh free function. """
        self.bm.free()

    # Add a custom method to directly access the internal bmesh:
    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.bm, item)
    
    def ensure_lookup_tables(self):
        self.verts.ensure_lookup_table()
        self.edges.ensure_lookup_table()
        self.faces.ensure_lookup_table()
        
    # def _retrieve_fc_given_fv(self, face, vertex):
    #         """
    #         Return the facecorner (loop) of face and vertex, None if not found
    #         """
    #         # for fc in self.vert2fc[vertex]:
    #         #         if fc.face == face:
    #         #             return fc
    #         # return None
    #         return self.face_vertex_to_facecorner[face][vertex]

    # def _retrieve_internal_angles_and_other_vertices(self, f, vi):
    #     vj, vk = [other for other in f.verts if other!=vi]
            
    #     fci = self._retrieve_fc_given_fv(f, vi)
    #     fcj = self._retrieve_fc_given_fv(f, vj)
    #     fck = self._retrieve_fc_given_fv(f, vk)

    #     # if fci is None or fcj is None or fck is None:
    #     #     raise ValueError(f"Unable to find a facecorner (loop) associated to vertex {vi.index}, {vj.index} or {vk.index} at face {f.index}")
    #     alpha_i = fci.calc_angle()
    #     alpha_j = fcj.calc_angle()
    #     alpha_k = fck.calc_angle()
    #     return vj, vk, alpha_i, alpha_j, alpha_k
    
    # def _calculate_cotan(self):
    #     self.facecorner_attributes["cotan"] = {fc:1/np.tan(fc.calc_angle()) for f in self.faces for fc in f.loops}

    def _calculate_corner_area(self):
        nfaces = len(self.faces)
        ffc = np.arange(3*nfaces).reshape((nfaces, 3)) # face corner index : face #f has corners [i, j, k];
        angles = self.internal_angles[ffc] # array of alpha_i, alpha_j, alpha_k
        eij = self.co[self.fv[:,1]] - self.co[self.fv[:,0]] # edges
        ejk = self.co[self.fv[:,2]] - self.co[self.fv[:,1]]
        eki = self.co[self.fv[:,0]] - self.co[self.fv[:,2]]
        lij2 = eij[:,0]*eij[:,0] + eij[:,1]*eij[:,1] + eij[:,2]*eij[:,2] # squared lengths of the edges
        ljk2 = ejk[:,0]*ejk[:,0] + ejk[:,1]*ejk[:,1] + ejk[:,2]*ejk[:,2]
        lki2 = eki[:,0]*eki[:,0] + eki[:,1]*eki[:,1] + eki[:,2]*eki[:,2]

        # in case triangle i,j,k is non obtuse (all its angles are < pi/2) then this is the area of each face corner
        non_obtuse_area_i = 0.125 * (lij2 * self.cotan[ffc[:,2]] + lki2 * self.cotan[ffc[:,1]])
        non_obtuse_area_j = 0.125 * (ljk2 * self.cotan[ffc[:,0]] + lij2 * self.cotan[ffc[:,2]])
        non_obtuse_area_k = 0.125 * (lki2 * self.cotan[ffc[:,1]] + ljk2 * self.cotan[ffc[:,0]])

        facecorners_areas = np.zeros(3*nfaces).reshape((nfaces, 3))
        # check whether the angles are less than pi/2
        small_angle_bool = angles < np.pi/2
        big_angle_bool = ~small_angle_bool
        # True for all corners of triangle i,j,k ==> use the non_obtuse_area
        non_obtuse_bool = np.logical_and(small_angle_bool[:,0], np.logical_and(small_angle_bool[:,1], small_angle_bool[:,2]))
        facecorners_areas[non_obtuse_bool, 0] = non_obtuse_area_i[non_obtuse_bool]
        facecorners_areas[non_obtuse_bool, 1] = non_obtuse_area_j[non_obtuse_bool]
        facecorners_areas[non_obtuse_bool, 2] = non_obtuse_area_k[non_obtuse_bool]
        # False for corner p in {i,j,k}: use half the face area for p and a quarter for the two others corners
        facecorners_areas[big_angle_bool[:,0]] = self.face_areas[big_angle_bool[:,0]][:,None] * np.array([[0.5, 0.25, 0.25]])
        facecorners_areas[big_angle_bool[:,1]] = self.face_areas[big_angle_bool[:,1]][:,None] * np.array([[0.25, 0.5, 0.25]])
        facecorners_areas[big_angle_bool[:,2]] = self.face_areas[big_angle_bool[:,2]][:,None] * np.array([[0.25, 0.25, 0.5]])
        self.facecorner_attributes["area"] = facecorners_areas.ravel()

    def _calculate_corner_angles_and_face_areas(self):
        self.co = np.array([v.co for v in self.verts])

        vi = self.co[self.fv[:,0]]
        vj = self.co[self.fv[:,1]]
        vk = self.co[self.fv[:,2]]

        eij, ejk, eki = vj-vi, vk-vj, vi-vk
        lij2 = eij[:,0]*eij[:,0] + eij[:,1]*eij[:,1] + eij[:,2]*eij[:,2]
        ljk2 = ejk[:,0]*ejk[:,0] + ejk[:,1]*ejk[:,1] + ejk[:,2]*ejk[:,2]
        lki2 = eki[:,0]*eki[:,0] + eki[:,1]*eki[:,1] + eki[:,2]*eki[:,2]
        lij, ljk, lki = np.sqrt(lij2), np.sqrt(ljk2), np.sqrt(lki2)

        s = 0.5 * (lij + ljk + lki) # half perimeter of every triangle
        self.face_areas = 0.5 * np.sqrt(s * (s - lij) * (s - ljk) * (s - lki)) # Heron's formula for the area of the triangles
        
        q_i = -ljk2 + lij2 + lki2
        q_j = -lki2 + ljk2 + lij2
        q_k = -lij2 + lki2 + ljk2

        denom_inv = 1/(8*self.face_areas)
        self.cotan = np.zeros(3*len(bm.faces))
        self.cotan[0::3] = q_i*denom_inv
        self.cotan[1::3] = q_j*denom_inv
        self.cotan[2::3] = q_k*denom_inv

        self.internal_angles = np.zeros(3*len(bm.faces))
        self.internal_angles[0::3] = np.arccos(np.clip(q_i / (2*lij * lki), -1, 1))
        self.internal_angles[1::3] = np.arccos(np.clip(q_j / (2*ljk * lij), -1, 1))
        self.internal_angles[2::3] = np.arccos(np.clip(q_k / (2*lki * ljk), -1, 1))

    def calculate_required_data(self):

        self._calculate_corner_angles_and_face_areas()
        self._calculate_corner_area()
        self._calculate_vertex_area()
        self._calculate_vertex_basis()

    def _calculate_vertex_area(self):
        # self.vertex_attributes["area"] = np.array([np.sum([self.facecorner_attributes["area"][fc.index] for fc in self.vert2facecorner[v]]) for v in self.verts])
        val = []
        for v in self.verts:
            s = 0
            for fc in self.vert2facecorner[v]:
                s+= self.facecorner_attributes["area"][fc.index]
            val.append(s)
        self.vertex_attributes["area"] = np.array(val)
            

    def _calculate_vertex_basis(self):
        eij = np.array([v.link_edges[0].other_vert(v).co - v.co for v in bm.verts])

        n = np.array([v.normal for v in self.verts])
        u = eij - np.sum(eij*n, axis=1)[:,None]
        u = u/np.linalg.norm(u, axis=1)[:,None]
        v = np.cross(n, u)
        v = v/np.linalg.norm(v, axis=1)[:,None]

        self.vertex_attributes["u"] = u
        self.vertex_attributes["v"] = v


    # def _calculate_vertex_area_and_basis_vector(self, vi):
    #     """
    #     Calculate the mixed area at vertex vi (and its face corners)
    #     """
    #     vi_area = 0
    #     for f in vi.link_faces:

    #         vj, vk, alpha_i, alpha_j, alpha_k = self._retrieve_internal_angles_and_other_vertices(f, vi)

    #         if alpha_i < np.pi/2 and alpha_j < np.pi/2 and alpha_k < np.pi/2: # Non obtuse triangle
    #             eij = vj.co - vi.co
    #             eik = vk.co - vi.co
    #             fc_area = (1/8) * (dot_product(eij, eij) / np.tan(alpha_k) + dot_product(eik, eik) / np.tan(alpha_j))
    #         elif alpha_i > np.pi/2:
    #             fc_area = f.calc_area()/2
    #         else:
    #             fc_area = f.calc_area()/4

    #         fci = self._retrieve_fc_given_fv(f, vi)
    #         # if self.facecorner_attributes["area"][fci] != 0:
    #         #     raise ValueError("wtf")
    #         self.facecorner_attributes["area"][fci] = fc_area
    #         vi_area += fc_area

    #     eij = np.array(f.edges[0].verts[1].co - f.edges[0].verts[0].co)
    #     n = vi.normal
    #     # u = eij - (n[0] * eij[0] + n[1] * eij[1] + n[2] * eij[2])*n
    #     u = eij - dot_product(n, eij) * n
    #     # u = eij - np.dot(n, eij)*n
    #     u = u/np.linalg.norm(u)
    #     # v = np.cross(n, u) # slow !!
    #     # v = np.array([
    #     #    n[1]*u[2] - n[2]*u[1],
    #     #    n[2]*u[0] - n[0]*u[2],
    #     #    n[0]*u[1] - n[1]*u[0]
    #     # ])
    #     v = cross_product(n, u)
    #     v = v/np.linalg.norm(v)

    #     self.vertex_attributes["u"][vi] = u
    #     self.vertex_attributes["v"][vi] = v

    #     return vi_area
    
    # # def calculate_vertices_area_and_basis_vector(self):
    #     self.vertex_attributes["area"] = {v:self._calculate_vertex_area_and_basis_vector(v) for v in self.verts}

    # def calculate_curvature(self):
    #     self.vertex_attributes["SFM"] = {v:np.zeros(3) for v in self.verts}
    #     for face in self.faces:
    #         nf = np.array(face.normal)
    #         uf = np.array(face.edges[0].verts[1].co - face.edges[0].verts[0].co)
    #         uf = uf/np.linalg.norm(uf)
    #         vf = cross_product(nf, uf)

    #         vi, vj, vk = face.verts
    #         ni, nj, nk = vi.normal, vj.normal, vk.normal
    #         ejk, eki, eij = vk.co-vj.co, vi.co-vk.co, vj.co-vi.co
    #         vec_b = np.array([
    #             dot_product(nk-nj, uf),
    #             dot_product(nk-nj, vf),
    #             dot_product(ni-nk, uf),
    #             dot_product(ni-nk, vf),
    #             dot_product(nj-ni, uf),
    #             dot_product(nj-ni, vf),
    #         ])
    #         a, b = dot_product(ejk, uf), dot_product(ejk, vf)
    #         c, d = dot_product(eki, uf), dot_product(eki, vf)
    #         e, f = dot_product(eij, uf), dot_product(eij, vf)
    #         A = np.array([
    #             [a, b, 0],
    #             [0, a, b],
    #             [c, d, 0],
    #             [0, c, d],
    #             [e, f, 0],
    #             [0, e, f]
    #         ])
    #         x, _, _, _ = np.linalg.lstsq(A, vec_b, rcond=None)

    #         for j in range(3):
    #             fc = face.loops[j]
    #             vertex = fc.vert
    #             weight = self.facecorner_attributes["area"][fc.index] / self.vertex_attributes["area"][vertex.index]
    #             up, vp = self.vertex_attributes["u"][vertex.index], self.vertex_attributes["v"][vertex.index]

    #             ku, kuv, kv = project_curvature_tensor(uf, vf, nf, x[0], x[1], x[2], up, vp, np.array(vertex.normal))
    #             self.vertex_attributes["SFM"][vertex] = self.vertex_attributes["SFM"][vertex] + weight * np.array([ku, kuv, kv]) 


    #     self.vertex_attributes["k1"] = {}
    #     self.vertex_attributes["k2"] = {}
    #     self.vertex_attributes["e1"] = {}
    #     self.vertex_attributes["e2"] = {}
    #     for vertex in self.verts:
    #         # np_vector = v.normal
    #         up, vp = self.vertex_attributes["u"][vertex.index], self.vertex_attributes["v"][vertex.index]
    #         # up, vp = rotate_coord_system(up, vp, np.array(vertex.normal))

    #         ku, kuv, kv = self.vertex_attributes["SFM"][vertex]
    #         c, s, tt = 1, 0, 0 # cos, sin, tan^2
            
    #         if kuv != 0:
    #             # Jacobi rotation to diagonalize the second fundamental matrix
    #             h = 0.5 * (kv - ku) / kuv
    #             if h < 0:
    #                 tt = 1 / (h - np.sqrt(1 + h * h))
    #             else:
    #                 tt = 1 / (h + np.sqrt(1 + h * h))

    #             # Calculate cosine and sine for the rotation
    #             c = 1 / np.sqrt(1 + tt * tt)
    #             s = tt * c

    #         # Compute the principal curvatures
    #         k1 = ku - tt * kuv
    #         k2 = kv + tt * kuv

    #         # Determine the direction of the principal components
    #         if abs(k1) >= abs(k2):
    #             e1 = c * up - s * vp
    #         else:
    #             # Swap the principal curvatures if necessary
    #             k1, k2 = k2, k1
    #             e1 = s * up + c * vp
    #         e2 = cross_product(vertex.normal, e1)

    #         self.vertex_attributes["k1"][vertex]=k1
    #         self.vertex_attributes["k2"][vertex]=k2
    #         self.vertex_attributes["e1"][vertex]=e1
    #         self.vertex_attributes["e2"][vertex]=e2

    def _custom_solve_Ab_vectorized(self, a, b, c, d, e, f, vec_b):
        """
        Heavily optimised method to solve for Ax = vec_b (least square) (about 45x faster than scipy.sparse.linalg.lsqr(A, vec_b)). A is of the form:
        A = [
        A_block_0, 0, 0, ...
        0, A_block_1, 0, ...
        0, 0, A_block_2, ...
        ]
        with
        A_block_i = [
        [a[i], b[i], 0],
        [0, a[i], b[i]],
        [c[i], d[i], 0],
        [0, c[i], d[i]],
        [e[i], f[i], 0],
        [0, e[i], f[i]]
        ]
        One knows that min||Ax-vec_b||^2 is obtained for x = ((A.T @ A)^{-1}) @ A.T @ vec_b.
        Let's first compute (A.T @ A)^{-1} (AtAinv in the code)
        then A.T @ vec_b (Atvec_b in the code)
        before returning x
        """
        aa, bb, cc, dd, ee, ff = a*a, b*b, c*c, d*d, e*e, f*f
        aaa, bbb, ccc, ddd, eee, fff = aa*a, bb*b, cc*c, dd*d, ee*e, ff*f
        aaaa, bbbb, cccc, dddd, eeee, ffff = aaa*a, bbb*b, ccc*c, ddd*d, eee*e, fff*f

        denom = aaaa*dd + aaaa*ff - 2*aaa*b*c*d - 2*aaa*b*e*f + aa*bb*cc + aa*bb*dd + aa*bb*ee + aa*bb*ff + aa*cc*dd + 2*aa*cc*ff - 2*aa*c*d*e*f + aa*dddd + 2*aa*dd*ee + 2*aa*dd*ff + aa*ee*ff + aa*ffff - 2*a*bbb*c*d - 2*a*bbb*e*f - 2*a*b*ccc*d - 2*a*b*cc*e*f - 2*a*b*c*ddd - 2*a*b*c*d*ee - 2*a*b*c*d*ff - 2*a*b*dd*e*f - 2*a*b*eee*f - 2*a*b*e*fff + bbbb*cc + bbbb*ee + bb*cccc + bb*cc*dd + 2*bb*cc*ee + 2*bb*cc*ff - 2*bb*c*d*e*f + 2*bb*dd*ee + bb*eeee + bb*ee*ff + cccc*ff - 2*ccc*d*e*f + cc*dd*ee + cc*dd*ff + cc*ee*ff + cc*ffff - 2*c*ddd*e*f - 2*c*d*eee*f - 2*c*d*e*fff + dddd*ee + dd*eeee + dd*ee*ff
        m00 = aa*dd + aa*ff - 2*a*b*c*d - 2*a*b*e*f + bbbb + bb*cc + 2*bb*dd + bb*ee + 2*bb*ff + cc*ff - 2*c*d*e*f + dddd + dd*ee + 2*dd*ff + ffff
        m01 = -a*bbb - a*b*dd - a*b*ff - bb*c*d - bb*e*f - c*ddd - c*d*ff - dd*e*f - e*fff
        m02 = aa*bb + 2*a*b*c*d + 2*a*b*e*f + cc*dd + 2*c*d*e*f + ee*ff
        # m10 = m01
        m11 = aa*bb + aa*dd + aa*ff + bb*cc + bb*ee + cc*dd + cc*ff + dd*ee + ee*ff
        m12 = -aaa*b - aa*c*d - aa*e*f - a*b*cc - a*b*ee - ccc*d - cc*e*f - c*d*ee - eee*f
        # m20 = m02
        # m21 = m12
        m22 = aaaa + 2*aa*cc + aa*dd + 2*aa*ee + aa*ff - 2*a*b*c*d - 2*a*b*e*f + bb*cc + bb*ee + cccc + 2*cc*ee + cc*ff - 2*c*d*e*f + dd*ee + eeee

        nvertex = denom.shape[0]
        denom_inv = 1./denom
        m00, m01, m02, m11, m12, m22 = m00*denom_inv, m01*denom_inv, m02*denom_inv, m11*denom_inv, m12*denom_inv, m22*denom_inv
        val = np.zeros(9*nvertex)
        val[0::9] = m00
        val[1::9] = m01
        val[2::9] = m02
        val[3::9] = m01
        val[4::9] = m11
        val[5::9] = m12
        val[6::9] = m02
        val[7::9] = m12
        val[8::9] = m22
        row = np.repeat(np.arange(3*nvertex), 3)
        col = np.array([0,1,2]*(3*nvertex)) + 3*np.repeat(np.arange(nvertex), 9)

        AtAinv = scipy.sparse.coo_array((val, (row, col))).tocsr()
  
        Atvec_b = np.zeros(3*nvertex)
        Atvec_b[0::3] = a * vec_b[0::6] + c * vec_b[2::6] + e * vec_b[4::6]
        Atvec_b[1::3] = b * vec_b[0::6] + a * vec_b[1::6] + d * vec_b[2::6] + c * vec_b[3::6] + f * vec_b[4::6] + e * vec_b[5::6]
        Atvec_b[2::3] = b * vec_b[1::6] + d * vec_b[3::6] + f * vec_b[5::6]

        x = AtAinv.dot(Atvec_b) # Ax = vec_b least square : min||Ax - vec_b||^2 ==> x = (A.T * A)^{-1} * A.T * vec_b

        return x

    def calculate_curvature_vectorized(self, use_optimized_solve=True):

        nface = len(self.faces)
        nvertex = len(self.verts)

        # co = np.array([v.co for v in self.verts])
        nv = np.array([v.normal for v in self.verts])
        nf = np.array([f.normal for f in self.faces])
        # fv = np.array([[v.index for v in f.verts] for f in self.faces])

        uf = self.co[self.fv[:,0]]-self.co[self.fv[:,1]]
        uf = uf/np.linalg.norm(uf, axis=1)[:,None]
        vf = np.cross(nf, uf)

        vi, vj, vk = self.co[self.fv[:,0]], self.co[self.fv[:,1]], self.co[self.fv[:,2]]
        ni, nj, nk = nv[self.fv[:,0]], nv[self.fv[:,1]], nv[self.fv[:,2]]
        
        ejk, eki, eij = vk-vj, vi-vk, vj-vi
        njk, nki, nij = nk-nj, ni-nk, nj-ni

        a, b = np.einsum('ij,ij->i', ejk, uf), np.einsum('ij,ij->i', ejk, vf)
        c, d = np.einsum('ij,ij->i', eki, uf), np.einsum('ij,ij->i', eki, vf)
        e, f = np.einsum('ij,ij->i', eij, uf), np.einsum('ij,ij->i', eij, vf)

        vec_b = np.zeros(6*nface)
        vec_b[0::6] = np.einsum('ij,ij->i', njk, uf)
        vec_b[1::6] = np.einsum('ij,ij->i', njk, vf)
        vec_b[2::6] = np.einsum('ij,ij->i', nki, uf)
        vec_b[3::6] = np.einsum('ij,ij->i', nki, vf)
        vec_b[4::6] = np.einsum('ij,ij->i', nij, uf)
        vec_b[5::6] = np.einsum('ij,ij->i', nij, vf)

        if not use_optimized_solve:
            value = np.vstack((a,b, a,b, c,d, c,d, e,f, e,f)).T
            row = np.repeat(np.arange(6*nface),2)
            col = np.tile([0, 1, 1, 2], 3*len(bm.faces)).reshape((-1,4)) + 3*np.repeat(np.arange(len(bm.faces)), 3).reshape((-1,1))
            value = value.ravel()
            col = col.ravel()
            A = scipy.sparse.coo_array((value, (row, col))).tocsr()
            x = scipy.sparse.linalg.lsqr(A, vec_b)[0] # only return the solution
        else:
            x = self._custom_solve_Ab_vectorized(a, b, c, d, e, f, vec_b) # Heavily optimized code to speed up the computation : about 45x faster than scipy.sparse.linalg.lsqr(A, vec_b)
        # project X into the vertex frame
        weights = self.facecorner_attributes["area"] / self.vertex_attributes["area"][self.fv.ravel()]
        fv_ravel = self.fv.ravel()
        uv = self.vertex_attributes["u"]
        vv = self.vertex_attributes["v"]
        uv_dup = uv[fv_ravel]
        vv_dup = vv[fv_ravel]

        nv_dup = nv[fv_ravel].reshape((3*self.fv.shape[0], 3))
        nf_dup = np.tile(nf,3).reshape((3*nface, 3))
        uf_dup = np.tile(uf,3).reshape((3*nface, 3))
        vf_dup = np.tile(vf,3).reshape((3*nface, 3))
        x_dup = np.tile(x.reshape((nface, 3)), 3).reshape((3*nface, 3))
        new_ku, new_kuv, new_kv = project_curvature_tensor_vectorized(uf_dup, vf_dup, nf_dup, x_dup[:,0], x_dup[:,1], x_dup[:,2], uv_dup, vv_dup, nv_dup)
        # new_k is on the facecorner domain. weight it to express it on the vertex domain
        # Build a matrix of mass so that M * new_k is on the vertex domain
        row, col, val = [], [], []
        for v in self.verts:
            for fc in self.vert2facecorner[v]:
                row.append(v.index)
                col.append(fc.index)
                val.append(weights[fc.index])

        M = scipy.sparse.coo_array((val, (row, col))).tocsr()
        ku = M.dot(new_ku)
        kuv = M.dot(new_kuv)
        kv = M.dot(new_kv)
        # sfm = np.vstack((ku, kuv, kv)).T # curvature matrix is [[ku, kuv], [kuv, kv]] for each [ku, kuv, kv] in sfm

        c, s, tt = np.ones(nvertex), np.zeros(nvertex), np.zeros(nvertex) 
        h = 0.5 * (kv-ku)/kuv
        root = np.sqrt(1 + h*h)
        hneg = h<0
        hpos = ~hneg
        tt[hneg] = 1/(h[hneg]-root[hneg])
        tt[hpos] = 1/(h[hpos]+root[hpos])
        c = 1 / np.sqrt(1 + tt * tt)
        s = tt * c
        ttkuv = tt * kuv
        k1 = ku - ttkuv
        k2 = kv + ttkuv
        kuv0 = kuv == 0
        k1[kuv0] = ku[kuv0]
        k2[kuv0] = kv[kuv0]
        k1lk2 = np.abs(k1) < np.abs(k2)
        # abs(k1) > abs(k2) ==> e1 = cos * uv - sin * vv
        # else ==> e1 = sin * uv + cos * vp
        c, s = c[:,None], s[:,None]
        e1 = c * uv - s * vv
        e1[k1lk2] = s[k1lk2]*uv[k1lk2] + c[k1lk2] * vv[k1lk2]
        e2 = np.cross(nv, e1)
        e2 = e2 / np.linalg.norm(e2, axis=1)[:,None]
        return k1, k2, e1, e2

obj = bpy.data.objects["Suzanne"]

bpy.ops.object.mode_set(mode='OBJECT')
mesh = obj.data

bm = MYMesh()
bm.from_mesh(mesh)
bm.ensure_lookup_tables()
bm.calculate_required_data()
k1, k2, e1, e2 = bm.calculate_curvature_vectorized()


In [75]:
%timeit bm.calculate_curvature_vectorized(truc=False)
%timeit bm.calculate_curvature_vectorized(truc=True)


204 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
215 ms ± 4.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
def tmp():
    bm = MYMesh()
    bm.from_mesh(mesh)

%timeit tmp()
%timeit bm.ensure_lookup_tables()
%timeit bm.calculate_required_data()
%timeit bm.calculate_curvature_vectorized()

def tmp():
    bm = MYMesh()
    bm.from_mesh(mesh)
    bm.ensure_lookup_tables()
    bm.calculate_required_data()
    k1, k2, e1, e2 = bm.calculate_curvature_vectorized()

%timeit tmp()

91.5 ms ± 2.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
3.65 μs ± 79.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
126 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
214 ms ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
420 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [80]:
214+126+92

432

: 

In [77]:
%load_ext line_profiler

%lprun -f bm.calculate_curvature_vectorized bm.calculate_curvature_vectorized()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Timer unit: 1e-07 s

Total time: 0.387817 s
File: C:\Users\pierr\AppData\Local\Temp\ipykernel_11672\971596443.py
Function: calculate_curvature_vectorized at line 483

Line #      Hits         Time  Per Hit   % Time  Line Contents
   483                                               def calculate_curvature_vectorized(self, use_optimized_solve=True, truc=False):
   484                                           
   485         1        102.0    102.0      0.0          nface = len(self.faces)
   486         1         35.0     35.0      0.0          nvertex = len(self.verts)
   487                                           
   488                                                   # co = np.array([v.co for v in self.verts])
   489         1     424551.0 424551.0     10.9          nv = np.array([v.normal for v in self.verts])
   490         1     741901.0 741901.0     19.1          nf = np.array([f.normal for f in self.faces])
   491                                                   # fv = np

In [60]:
np.random.seed(3)
tmpA = np.random.rand(3*1000000).reshape((-1, 3))
tmpB = np.random.rand(3*1000000).reshape((-1, 3))

%timeit (tmpA[:,None,:] @ tmpB[...,None]).ravel()
%timeit np.sum(tmpA*tmpB, axis=1)
%timeit np.einsum('ij,ij->i',tmpA,tmpB)
%timeit (tmpA*tmpB).sum(1)

9.96 ms ± 177 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
31.1 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
9.03 ms ± 422 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
30.9 ms ± 282 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
if "e1" in mesh.attributes:
    mesh.attributes.remove(mesh.attributes["e2"])
    mesh.attributes.remove(mesh.attributes["e1"])

attr = mesh.attributes.new(name="e1", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector',e1.flatten())

attr = mesh.attributes.new(name="e2", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector',e2.flatten())

attr = mesh.attributes.new(name="k1", type='FLOAT', domain='POINT')
attr.data.foreach_set('value', k1)

attr = mesh.attributes.new(name="k2", type='FLOAT', domain='POINT')
attr.data.foreach_set('value', k2)
    
# attr = mesh.attributes.new(name="new_e1", type='FLOAT_VECTOR', domain='POINT')
# attr.data.foreach_set('vector',e1.flatten())

# attr = mesh.attributes.new(name="new_e2", type='FLOAT_VECTOR', domain='POINT')
# attr.data.foreach_set('vector',e2.flatten())

In [ ]:
nf = np.array([f.normal for f in bm.faces])
weights = np.random.rand(15744*3)

fv = np.array([[v.index for v in f.verts] for f in bm.faces])
fv_ravel = fv.ravel()
print(fv_ravel)
def my_func(self):
    uv_dup = np.array([self.vertex_attributes["u"][self.verts[i]] for i in fv_ravel])
    return uv_dup

def my_other_func(self):
    uv = np.array(list(self.vertex_attributes["u"].values()))
    uv_dup = uv[fv_ravel] #np.array([uv[i] for i in fv_ravel])
    return uv, uv_dup
my_func(bm)
%timeit my_func(bm)
%timeit my_other_func(bm)

In [ ]:
def calc_face_normals(FV):
    """
    Calculate the normal vectors for each face of a mesh.
    
    Parameters:
    FV (dict): A dictionary representing a triangle mesh in face-vertex structure.
               It should have keys 'vertices' (Nv x 3 matrix) and 'faces' (Nf x 3 matrix).
    
    Returns:
    FaceNormals (numpy.ndarray): (Nf x 3) matrix containing the normal vector at each face.
    """
    # Get all edge vectors
    e0 = FV['vertices'][FV['faces'][:, 2], :] - FV['vertices'][FV['faces'][:, 1], :]
    e1 = FV['vertices'][FV['faces'][:, 0], :] - FV['vertices'][FV['faces'][:, 2], :]

    # Calculate normal of each face using the cross product of two edges
    FaceNormals = np.cross(e0, e1)

    # Normalize the face normals
    FaceNormals /= np.linalg.norm(FaceNormals, axis=1)[:, np.newaxis]

    return FaceNormals

def calc_vertex_normals(FV, N):
    """
    Calculate vertex normals and voronoi areas for each vertex.
    
    Parameters:
    FV (dict): A dictionary representing a triangle mesh in face-vertex structure. 
               It should have keys 'vertices' (Nv x 3 matrix) and 'faces' (Nf x 3 matrix).
    N (numpy.ndarray): Face normals, an (Nf x 3) matrix.
    
    Returns:
    VertexNormals (numpy.ndarray): (Nv x 3) matrix of normals at each vertex.
    Avertex (numpy.ndarray): (Nv x 1) voronoi area at each vertex.
    Acorner (numpy.ndarray): (Nf x 3) slice of the voronoi area at each face corner.
    up (numpy.ndarray): (Nv x 3) part of the initial vertex coordinate system.
    vp (numpy.ndarray): (Nv x 3) part of the initial vertex coordinate system.
    """
    print('Calculating vertex normals... Please wait')

    # Get all edge vectors
    e0 = FV['vertices'][FV['faces'][:, 2], :] - FV['vertices'][FV['faces'][:, 1], :]
    e1 = FV['vertices'][FV['faces'][:, 0], :] - FV['vertices'][FV['faces'][:, 2], :]
    e2 = FV['vertices'][FV['faces'][:, 1], :] - FV['vertices'][FV['faces'][:, 0], :]
    # Normalize edge vectors
    e0_norm = e0 / np.linalg.norm(e0, axis=1)[:, np.newaxis]
    e1_norm = e1 / np.linalg.norm(e1, axis=1)[:, np.newaxis]
    e2_norm = e2 / np.linalg.norm(e2, axis=1)[:, np.newaxis]

    # Calculate edge lengths
    de0 = np.linalg.norm(e0, axis=1)
    de1 = np.linalg.norm(e1, axis=1)
    de2 = np.linalg.norm(e2, axis=1)
    l2 = np.vstack((de0**2, de1**2, de2**2)).T

    # Calculate weights for the voronoi area calculation
    ew = np.vstack((l2[:, 0] * (l2[:, 1] + l2[:, 2] - l2[:, 0]),
                    l2[:, 1] * (l2[:, 2] + l2[:, 0] - l2[:, 1]),
                    l2[:, 2] * (l2[:, 0] + l2[:, 1] - l2[:, 2]))).T

    # Heron's formula for triangle area
    s = (de0 + de1 + de2) / 2
    Af = np.sqrt(s * (s - de0) * (s - de1) * (s - de2))

    # Initialize outputs
    Acorner = np.zeros((FV['faces'].shape[0], 3))
    Avertex = np.zeros((FV['vertices'].shape[0],))
    VertexNormals = np.zeros((FV['vertices'].shape[0], 3))
    up = np.zeros((FV['vertices'].shape[0], 3))
    vp = np.zeros((FV['vertices'].shape[0], 3))

    # Calculate vertex normals and areas
    for i in range(FV['faces'].shape[0]):
        # Calculate weights according to N.Max [1999]
        wfv1 = Af[i] / (de1[i]**2 * de2[i]**2)
        wfv2 = Af[i] / (de0[i]**2 * de2[i]**2)
        wfv3 = Af[i] / (de1[i]**2 * de0[i]**2)

        # Accumulate weighted normals for each vertex
        VertexNormals[FV['faces'][i, 0], :] += wfv1 * N[i, :]
        VertexNormals[FV['faces'][i, 1], :] += wfv2 * N[i, :]
        VertexNormals[FV['faces'][i, 2], :] += wfv3 * N[i, :]

        # Calculate voronoi areas (Meyer et al. [2002])
        if ew[i, 0] <= 0:
            Acorner[i, 1] = -0.25 * l2[i, 2] * Af[i] / np.dot(e0[i, :], e2[i, :])
            Acorner[i, 2] = -0.25 * l2[i, 1] * Af[i] / np.dot(e0[i, :], e1[i, :])
            Acorner[i, 0] = Af[i] - Acorner[i, 1] - Acorner[i, 2]
        elif ew[i, 1] <= 0:
            Acorner[i, 2] = -0.25 * l2[i, 0] * Af[i] / np.dot(e1[i, :], e0[i, :])
            Acorner[i, 0] = -0.25 * l2[i, 2] * Af[i] / np.dot(e1[i, :], e2[i, :])
            Acorner[i, 1] = Af[i] - Acorner[i, 0] - Acorner[i, 2]
        elif ew[i, 2] <= 0:
            Acorner[i, 0] = -0.25 * l2[i, 1] * Af[i] / np.dot(e2[i, :], e1[i, :])
            Acorner[i, 1] = -0.25 * l2[i, 0] * Af[i] / np.dot(e2[i, :], e0[i, :])
            Acorner[i, 2] = Af[i] - Acorner[i, 0] - Acorner[i, 1]
        else:
            ewscale = 0.5 * Af[i] / (ew[i, 0] + ew[i, 1] + ew[i, 2])
            Acorner[i, 0] = ewscale * (ew[i, 1] + ew[i, 2])
            Acorner[i, 1] = ewscale * (ew[i, 0] + ew[i, 2])
            Acorner[i, 2] = ewscale * (ew[i, 0] + ew[i, 1])

        # Accumulate vertex areas
        Avertex[FV['faces'][i, 0]] += Acorner[i, 0]
        Avertex[FV['faces'][i, 1]] += Acorner[i, 1]
        Avertex[FV['faces'][i, 2]] += Acorner[i, 2]

        # Calculate initial coordinate system
        up[FV['faces'][i, 0], :] = e2_norm[i, :]
        up[FV['faces'][i, 1], :] = e0_norm[i, :]
        up[FV['faces'][i, 2], :] = e1_norm[i, :]

    # Normalize vertex normals
    VertexNormals /= np.linalg.norm(VertexNormals, axis=1)[:, np.newaxis]
    # Calculate initial vertex coordinate system
    for i in range(FV['vertices'].shape[0]):
        up[i, :] = np.random.rand(3)*2-1
        up[i, :] = np.cross(up[i, :], VertexNormals[i, :])
        up[i, :] /= np.linalg.norm(up[i, :])
        vp[i, :] = np.cross(VertexNormals[i, :], up[i, :])

    print('Finished calculating vertex normals')
    
    return VertexNormals, Avertex, Acorner, up, vp

def project_curvature_tensor(uf, vf, nf, old_ku, old_kuv, old_kv, up, vp):
    """
    Perform a projection of the tensor variables to the vertex coordinate system.
    
    Parameters:
    uf, vf (numpy.ndarray): Face coordinate system unit vectors.
    nf (numpy.ndarray): Normal vector of the face.
    old_ku, old_kuv, old_kv (float): Face curvature tensor variables.
    up, vp (numpy.ndarray): Vertex coordinate system unit vectors.
    
    Returns:
    new_ku, new_kuv, new_kv (float): Vertex curvature tensor variables.
    """
    
    # Rotate the coordinate system
    r_new_u, r_new_v = rotate_coord_system(up, vp, nf)
    
    # Curvature tensor matrix for the face
    old_tensor = np.array([[old_ku, old_kuv], 
                           [old_kuv, old_kv]])
    
    # Project coordinates
    u1 = np.dot(r_new_u, uf)
    v1 = np.dot(r_new_u, vf)
    u2 = np.dot(r_new_v, uf)
    v2 = np.dot(r_new_v, vf)

    # Calculate new curvature tensor variables
    new_ku = np.dot([u1, v1], np.dot(old_tensor, [u1, v1]))
    new_kuv = np.dot([u1, v1], np.dot(old_tensor, [u2, v2]))
    new_kv = np.dot([u2, v2], np.dot(old_tensor, [u2, v2]))

    return new_ku, new_kuv, new_kv

def rotate_coord_system_vectorized(u,v,nf):
    """
    rotate the vectors u and v to the plane of normal nf
    Args:
        u (np.array (N,3)) : 1st set of vector to be rotated
        v (np.array (N,3)) : 2nd set of vector to be rotated
        nf (np.array (N,3)) : normal vectors of the plane to rotate into
    Output:
        new_u, new_v (np.array(N, 3)) : rotated u and v (orthogonal to nf)
    """
    n = np.cross(u,v)
    n = n/np.linalg.norm(n, axis=1)[:,None]

    dot = np.sum(nf*n, axis=1)[:,None] # np.dot(nf,n) : nf.n
    perp = nf - dot*n
    dperp = (nf + n)/(1+dot)
    new_u = u - dperp * np.sum(perp*u, axis=1)[:,None] # u - dperp * (perp.u)
    new_v = v - dperp * np.sum(perp*v, axis=1)[:,None]

    threshold = np.where(dot<=-1)[0]
    new_u[threshold] = -u[threshold]
    new_v[threshold] = -v[threshold]

    return new_u, new_v


def rotate_coord_system(u,v,nf):
    """
    rotate the vectors u and v to the plane of normal nf
    Args:
        u (np.array (N,3)) : 1st set of vector to be rotated
        v (np.array (N,3)) : 2nd set of vector to be rotated
        nf (np.array (N,3)) : normal vectors of the plane to rotate into
    Output:
        new_u, new_v (np.array(N, 3)) : rotated u and v (orthogonal to nf)
    """
    n = np.cross(u,v)
    n = n/np.linalg.norm(n)

    dot = np.dot(nf,n)
    if dot <= -1:
        return -u,-v
    
    perp = nf - dot*n
    dperp = (nf + n)/(1+dot)
    new_u = u - dperp * np.dot(perp,u) # u - dperp * (perp.u)
    new_v = v - dperp * np.dot(perp,v)

    return new_u, new_v



def calc_curvature(FV, VertexNormals, FaceNormals, Avertex, Acorner, up, vp):
    """
    Calculate the second fundamental matrix and curvature using least squares.
    
    Parameters:
    FV (dict): Face-vertex data structure containing a list of vertices and a list of faces.
    VertexNormals (numpy.ndarray): n x 3 matrix (n = number of vertices) containing the normal at each vertex.
    FaceNormals (numpy.ndarray): m x 3 matrix (m = number of faces) containing the normal of each face.
    Avertex (numpy.ndarray): Voronoi area at each vertex.
    Acorner (numpy.ndarray): Voronoi area slice at each face corner.
    up (numpy.ndarray): Vertex coordinate system (u vector).
    vp (numpy.ndarray): Vertex coordinate system (v vector).
    
    Returns:
    FaceSFM (list): List of m elements containing the second fundamental matrix for each face.
    VertexSFM (list): List of n elements containing the second fundamental matrix for each vertex.
    wfp (numpy.ndarray): Corner Voronoi weights.
    """
    print('Calculating Curvature Tensors... Please wait')
    
    # Initialize the second fundamental matrix for faces and vertices
    FaceSFM = [np.zeros((2, 2)) for _ in range(FV['faces'].shape[0])]
    VertexSFM = [np.zeros((2, 2)) for _ in range(FV['vertices'].shape[0])]

    # Get all edge vectors
    e0 = FV['vertices'][FV['faces'][:, 2], :] - FV['vertices'][FV['faces'][:, 1], :]
    e1 = FV['vertices'][FV['faces'][:, 0], :] - FV['vertices'][FV['faces'][:, 2], :]
    e2 = FV['vertices'][FV['faces'][:, 1], :] - FV['vertices'][FV['faces'][:, 0], :]

    # Normalize edge vectors
    e0_norm = e0 / np.linalg.norm(e0, axis=1)[:, np.newaxis]
    # e1_norm = e1 / np.linalg.norm(e1, axis=1)[:, np.newaxis]
    # e2_norm = e2 / np.linalg.norm(e2, axis=1)[:, np.newaxis]

    wfp = np.zeros((FV['faces'].shape[0], 3))
    # Iterate over each face
    for i in range(FV['faces'].shape[0]):
        # Calculate Curvature Per Face
        nf = FaceNormals[i, :]
        t = e0_norm[i, :].T
        B = np.cross(nf, t).T
        B = B / np.linalg.norm(B)

        # Extract relevant normals in face vertices
        n0 = VertexNormals[FV['faces'][i, 0], :]
        n1 = VertexNormals[FV['faces'][i, 1], :]
        n2 = VertexNormals[FV['faces'][i, 2], :]

        # Solve least squares problem of the form Ax = b
        # A = np.array([
        #     [np.dot(e0[i, :], t), np.dot(e0[i, :], B), 0],
        #     [0, np.dot(e0[i, :], t), np.dot(e0[i, :], B)],
        #     [np.dot(e1[i, :], t), np.dot(e1[i, :], B), 0],
        #     [0, np.dot(e1[i, :], t), np.dot(e1[i, :], B)],
        #     [np.dot(e2[i, :], t), np.dot(e2[i, :], B), 0],
        #     [0, np.dot(e2[i, :], t), np.dot(e2[i, :], B)]
        # ])
        
        b = np.array([
            np.dot(n2 - n1, t),
            np.dot(n2 - n1, B),
            np.dot(n0 - n2, t),
            np.dot(n0 - n2, B),
            np.dot(n1 - n0, t),
            np.dot(n1 - n0, B)
        ])

        # Solve the system A * x = b
        # x,_,_,_ = np.linalg.lstsq(A, b, rcond=None)

        x = custom_solve_Ab(a=np.dot(e0[i, :], t), b=np.dot(e0[i, :], B), c=np.dot(e1[i, :], t), d=np.dot(e1[i, :], B), e=np.dot(e2[i, :], t), f=np.dot(e2[i, :], B), vec_b=b)

        # Construct the face second fundamental matrix
        FaceSFM[i] = np.array([[x[0], x[1]], [x[1], x[2]]])
        
        # Calculate Curvature Per Vertex
        wfp[i, 0] = Acorner[i, 0] / Avertex[FV['faces'][i, 0]]
        wfp[i, 1] = Acorner[i, 1] / Avertex[FV['faces'][i, 1]]
        wfp[i, 2] = Acorner[i, 2] / Avertex[FV['faces'][i, 2]]
        
        # Calculate new coordinate system and project the tensor
        for j in range(3):
            new_ku, new_kuv, new_kv = project_curvature_tensor(
                t, B, nf, x[0], x[1], x[2], 
                up[FV['faces'][i, j], :], 
                vp[FV['faces'][i, j], :]
            )
            VertexSFM[FV['faces'][i, j]] += wfp[i, j] * np.array([[new_ku, new_kuv], [new_kuv, new_kv]])

    print('Finished Calculating Curvature Tensors.')
    return FaceSFM, VertexSFM, wfp


def custom_solve_Ab(a,b,c,d,e,f, vec_b):
    aa, bb, cc, dd, ee, ff = a*a, b*b, c*c, d*d, e*e, f*f
    aaa, bbb, ccc, ddd, eee, fff = aa*a, bb*b, cc*c, dd*d, ee*e, ff*f
    aaaa, bbbb, cccc, dddd, eeee, ffff = aaa*a, bbb*b, ccc*c, ddd*d, eee*e, fff*f

    denom = aaaa*dd + aaaa*ff - 2*aaa*b*c*d - 2*aaa*b*e*f + aa*bb*cc + aa*bb*dd + aa*bb*ee + aa*bb*ff + aa*cc*dd + 2*aa*cc*ff - 2*aa*c*d*e*f + aa*dddd + 2*aa*dd*ee + 2*aa*dd*ff + aa*ee*ff + aa*ffff - 2*a*bbb*c*d - 2*a*bbb*e*f - 2*a*b*ccc*d - 2*a*b*cc*e*f - 2*a*b*c*ddd - 2*a*b*c*d*ee - 2*a*b*c*d*ff - 2*a*b*dd*e*f - 2*a*b*eee*f - 2*a*b*e*fff + bbbb*cc + bbbb*ee + bb*cccc + bb*cc*dd + 2*bb*cc*ee + 2*bb*cc*ff - 2*bb*c*d*e*f + 2*bb*dd*ee + bb*eeee + bb*ee*ff + cccc*ff - 2*ccc*d*e*f + cc*dd*ee + cc*dd*ff + cc*ee*ff + cc*ffff - 2*c*ddd*e*f - 2*c*d*eee*f - 2*c*d*e*fff + dddd*ee + dd*eeee + dd*ee*ff
    m00 = aa*dd + aa*ff - 2*a*b*c*d - 2*a*b*e*f + bbbb + bb*cc + 2*bb*dd + bb*ee + 2*bb*ff + cc*ff - 2*c*d*e*f + dddd + dd*ee + 2*dd*ff + ffff
    m01 = -a*bbb - a*b*dd - a*b*ff - bb*c*d - bb*e*f - c*ddd - c*d*ff - dd*e*f - e*fff
    m02 = aa*bb + 2*a*b*c*d + 2*a*b*e*f + cc*dd + 2*c*d*e*f + ee*ff
    # m10 = m01
    m11 = aa*bb + aa*dd + aa*ff + bb*cc + bb*ee + cc*dd + cc*ff + dd*ee + ee*ff
    m12 = -aaa*b - aa*c*d - aa*e*f - a*b*cc - a*b*ee - ccc*d - cc*e*f - c*d*ee - eee*f
    # m20 = m02
    # m21 = m12
    m22 = aaaa + 2*aa*cc + aa*dd + 2*aa*ee + aa*ff - 2*a*b*c*d - 2*a*b*e*f + bb*cc + bb*ee + cccc + 2*cc*ee + cc*ff - 2*c*d*e*f + dd*ee + eeee

    Mt = np.array([
        [a,0,c,0,e,0],
        [b,a,d,c,f,e],
        [0,b,0,d,0,f]
        ])
    MtM_inv = (1/denom) * np.array([
        [m00, m01, m02],
        [m01, m11, m12],
        [m02, m12, m22]
    ])

    return MtM_inv @ Mt @ vec_b


def get_principal_curvatures(FV, VertexSFM, up, vp):
    """
    Calculate the principal curvatures and their directions from the second fundamental matrix.
    
    Parameters:
    FV (dict): Mesh face-vertex data structure.
    VertexSFM (list): Second fundamental matrix for each vertex.
    up (numpy.ndarray): Vertex local coordinate frame (u vector).
    vp (numpy.ndarray): Vertex local coordinate frame (v vector).
    
    Returns:
    PrincipalCurvatures (numpy.ndarray): 2 x [number of vertices] matrix containing the principal curvature values.
    PrincipalDir1 (numpy.ndarray): Direction vectors of the first principal component.
    PrincipalDir2 (numpy.ndarray): Direction vectors of the second principal component.
    """
    print('Calculating Principal Components')

    # Initialize arrays for principal curvatures and directions
    PrincipalCurvatures = np.zeros((2, FV['vertices'].shape[0]))
    PrincipalDir1 = np.zeros((FV['vertices'].shape[0], 3))
    PrincipalDir2 = np.zeros((FV['vertices'].shape[0], 3))

    # Iterate over each vertex
    for i in range(FV['vertices'].shape[0]):
        # Compute the normal vector np
        np_vector = np.cross(up[i, :], vp[i, :])

        # Rotate the coordinate system to align with the normal
        r_old_u, r_old_v = rotate_coord_system(up[i, :], vp[i, :], np_vector)
        # Retrieve components of the second fundamental matrix
        ku = VertexSFM[i][0, 0]
        kuv = VertexSFM[i][0, 1]
        kv = VertexSFM[i][1, 1]

        # Initialize rotation parameters
        c = 1
        s = 0
        tt = 0

        if kuv != 0:
            # Jacobi rotation to diagonalize the second fundamental matrix
            h = 0.5 * (kv - ku) / kuv
            if h < 0:
                tt = 1 / (h - np.sqrt(1 + h * h))
            else:
                tt = 1 / (h + np.sqrt(1 + h * h))

            # Calculate cosine and sine for the rotation
            c = 1 / np.sqrt(1 + tt * tt)
            s = tt * c

        # Compute the principal curvatures
        k1 = ku - tt * kuv
        k2 = kv + tt * kuv

        # Determine the direction of the principal components
        if abs(k1) >= abs(k2):
            PrincipalDir1[i, :] = c * r_old_u - s * r_old_v
        else:
            # Swap the principal curvatures if necessary
            k1, k2 = k2, k1
            PrincipalDir1[i, :] = s * r_old_u + c * r_old_v

        # Compute the second principal direction
        PrincipalDir2[i, :] = np.cross(np_vector, PrincipalDir1[i, :])

        # Store the principal curvature values
        PrincipalCurvatures[0, i] = k1
        PrincipalCurvatures[1, i] = k2

        # Check for NaN values
        if np.isnan(k1) or np.isnan(k2):
            print('NAN')

    print('Finished Calculating Principal Components')
    return PrincipalCurvatures, PrincipalDir1, PrincipalDir2


def project_c_tensor(uf, vf, nf, Old_C, up, vp):
    """
    Project the curvature tensor variables from the face coordinate system to the vertex coordinate system.
    
    Parameters:
    uf (numpy.ndarray): Face coordinate system (u vector).
    vf (numpy.ndarray): Face coordinate system (v vector).
    nf (numpy.ndarray): Face normal vector.
    Old_C (numpy.ndarray): Face curvature tensor variables as a vector.
    up (numpy.ndarray): Vertex coordinate system (u vector).
    vp (numpy.ndarray): Vertex coordinate system (v vector).
    
    Returns:
    new_C (numpy.ndarray): Vertex curvature tensor variables as a column vector.
    """
    
    # Initialize a tensor matrix (not used but keeping the placeholder for potential future use)
    new_CMatrix = np.zeros((2, 2, 2))

    # Rotate the coordinate system to align with the face normal
    r_new_u, r_new_v = rotate_coord_system(up, vp, nf)

    # Compute the projection coefficients
    u1 = np.dot(r_new_u, uf)
    v1 = np.dot(r_new_u, vf)
    u2 = np.dot(r_new_v, uf)
    v2 = np.dot(r_new_v, vf)

    # Calculate the new curvature tensor components
    new_C = np.zeros(4)
    new_C[0] = Old_C[0] * u1 * u1 * u1 + 3 * Old_C[1] * u1 * u1 * v1 + 3 * Old_C[2] * u1 * v1 * v1 + Old_C[3] * v1 * v1 * v1
    new_C[1] = (Old_C[0] * u2 * u1 * u1 +
                Old_C[1] * (v2 * u1 * u1 + 2 * u2 * u1 * v1) +
                Old_C[2] * (u2 * v1 * v1 + 2 * u1 * v1 * v2) +
                Old_C[3] * v2 * v1 * v1)
    new_C[2] = (Old_C[0] * u1 * u2 * u2 +
                Old_C[1] * (v1 * u2 * u2 + 2 * u2 * u1 * v2) +
                Old_C[2] * (u1 * v2 * v2 + 2 * u2 * v2 * v1) +
                Old_C[3] * v1 * v2 * v2)
    new_C[3] = Old_C[0] * u2 * u2 * u2 + 3 * Old_C[1] * u2 * u2 * v2 + 3 * Old_C[2] * u2 * v2 * v2 + Old_C[3] * v2 * v2 * v2

    return new_C



def calc_curvature_derivative(FV, FaceNormals, PrincipalCurvatures, up, vp, wfp):
    """
    Calculate the curvature derivative matrix (2x2x2) for each face and vertex in the mesh.
    
    Parameters:
    FV (dict): Face-vertex data structure containing a list of vertices and faces.
    FaceNormals (numpy.ndarray): Array containing the normals of each face.
    PrincipalCurvatures (numpy.ndarray): 2x[number of vertices] matrix with principal curvature values.
    up, vp (numpy.ndarray): Arrays defining the vertex local coordinate frames.
    wfp (numpy.ndarray): Corner Voronoi weights.
    
    Returns:
    FaceCMatrix (numpy.ndarray): m x (2x2x2) array with curvature derivative tensor for each face.
    VertexCMatrix (numpy.ndarray): n x (2x2x2) array with curvature derivative tensor for each vertex.
    Cmagnitude (numpy.ndarray): Curvature magnitude for each vertex.
    """

    print('Calculating C Tensors... Please wait')

    # Initialization
    FaceCMatrix = np.zeros((FV['faces'].shape[0], 4))
    VertexCMatrix = np.zeros((FV['vertices'].shape[0], 4))
    FV_SFM = [np.zeros((2, 2)) for _ in range(3)]
    # Cmagnitude = np.zeros(FV['vertices'].shape[0])

    # Get all edge vectors
    e0 = FV['vertices'][FV['faces'][:, 2], :] - FV['vertices'][FV['faces'][:, 1], :]
    e1 = FV['vertices'][FV['faces'][:, 0], :] - FV['vertices'][FV['faces'][:, 2], :]
    e2 = FV['vertices'][FV['faces'][:, 1], :] - FV['vertices'][FV['faces'][:, 0], :]

    # Normalize edge vectors
    e0_norm = e0 / np.linalg.norm(e0, axis=1)[:, np.newaxis]
    # e1_norm = e1 / np.linalg.norm(e1, axis=1)[:, np.newaxis]
    # e2_norm = e2 / np.linalg.norm(e2, axis=1)[:, np.newaxis]

    for i in range(FV['faces'].shape[0]):
        # Set face coordinate frame
        nf = FaceNormals[i, :]
        t = e0_norm[i, :].T
        B = np.cross(nf, t)
        B = B / np.linalg.norm(B)

        # Solve least squares problem Ax = b
        u = np.array([np.dot(e0[i, :], t), np.dot(e1[i, :], t), np.dot(e2[i, :], t)])
        v = np.array([np.dot(e0[i, :], B), np.dot(e1[i, :], B), np.dot(e2[i, :], B)])

        for j in range(3):
            np_vec = np.cross(up[FV['faces'][i, j], :], vp[FV['faces'][i, j], :])
            np_vec = np_vec / np.linalg.norm(np_vec)

            k1 = PrincipalCurvatures[0, FV['faces'][i, j]]
            k2 = PrincipalCurvatures[1, FV['faces'][i, j]]

            new_ku, new_kuv, new_kv = project_curvature_tensor(
                up[FV['faces'][i, j], :].T, vp[FV['faces'][i, j], :].T, np_vec, k1, 0, k2, t.T, B.T
            )
            FV_SFM[j] = np.array([[new_ku, new_kuv], [new_kuv, new_kv]])

        # Compute changes in curvature
        Delta_e = np.array([
            FV_SFM[2][0, 0] - FV_SFM[1][0, 0],
            FV_SFM[0][0, 0] - FV_SFM[2][0, 0],
            FV_SFM[1][0, 0] - FV_SFM[0][0, 0]
        ])
        Delta_f = np.array([
            FV_SFM[2][0, 1] - FV_SFM[1][0, 1],
            FV_SFM[0][0, 1] - FV_SFM[2][0, 1],
            FV_SFM[1][0, 1] - FV_SFM[0][0, 1]
        ])
        Delta_g = np.array([
            FV_SFM[2][1, 1] - FV_SFM[1][1, 1],
            FV_SFM[0][1, 1] - FV_SFM[2][1, 1],
            FV_SFM[1][1, 1] - FV_SFM[0][1, 1]
        ])

        # Least squares matrix setup
        sumU2 = np.dot(u, u)
        sumV2 = np.dot(v, v)
        sumUV = np.dot(u, v)
        A = np.array([
            [sumU2, sumUV, 0, 0],
            [sumUV, 2 * sumU2 + sumV2, 2 * sumUV, 0],
            [0, 2 * sumUV, sumU2 + 2 * sumV2, sumUV],
            [0, 0, sumUV, sumV2]
        ])
        b = np.array([
            np.dot(Delta_e, u), 
            np.dot(Delta_e, v) + 2 * np.dot(Delta_f, u), 
            2 * np.dot(Delta_f, v) + np.dot(Delta_g, u), 
            np.dot(Delta_g, v)
        ])
        x,_,_,_ = np.linalg.lstsq(A, b, rcond=None)
        # x = np.linalg.solve(A, b)
        FaceCMatrix[i, :] = x

        # Project the tensor for each vertex and update the VertexCMatrix
        for j in range(3):
            new_CMatrix = project_c_tensor(t, B, nf, x, up[FV['faces'][i, j], :], vp[FV['faces'][i, j], :])
            VertexCMatrix[FV['faces'][i, j], :] += wfp[i, j] * new_CMatrix

    # Calculate the magnitude of the curvature derivative
    a = VertexCMatrix[:, 0]
    b = VertexCMatrix[:, 1]
    c = VertexCMatrix[:, 2]
    d = VertexCMatrix[:, 3]


    Cmagnitude = a ** 2 + 3 * b ** 2 + 3 * c ** 2 + d ** 2
    epsilon = 1e-5
    Cmagnitude[Cmagnitude < epsilon] = 0

    print('Finished Calculating C Tensors.')

    return FaceCMatrix, VertexCMatrix, Cmagnitude


FV = {'vertices':np.array([v.co for v in bm.verts]), "faces":np.array([[v.index for v in f.verts] for f in bm.faces])}

def get_face_normals(mesh:MYMesh):
    return np.array([f.normal for f in mesh.faces])



face_n = get_face_normals(bm)
VertexNormals, Avertex, Acorner, up, vp = calc_vertex_normals(FV, face_n)
# up, vp = np.array([bm.vertex_attributes["u"][v] for v in bm.verts]), np.array([bm.vertex_attributes["v"][v] for v in bm.verts])
# VertexNormals = np.array([v.normal for v in bm.verts])
# Avertex = [bm.vertex_attributes["area"][v] for v in bm.verts]
# Acorner = np.array([[bm.facecorner_attributes["area"][fc] for fc in f.loops] for f in bm.faces])
FaceSFM, VertexSFM, wfp = calc_curvature(FV, VertexNormals, face_n, Avertex, Acorner, up, vp)
PrincipalCurvatures, PrincipalDir1, PrincipalDir2 = get_principal_curvatures(FV, VertexSFM, up, vp)
# FaceCMatrix, VertexCMatrix, Cmagnitude = calc_curvature_derivative(FV, face_n, PrincipalCurvatures, up, vp, wfp)


In [ ]:
if "debug_e1" in mesh.attributes:
    mesh.attributes.remove(mesh.attributes["debug_e2"])
    mesh.attributes.remove(mesh.attributes["debug_e1"])

attr = mesh.attributes.new(name="debug_e1", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector',PrincipalDir1.flatten())

attr = mesh.attributes.new(name="debug_e2", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector', PrincipalDir2.flatten())
        